# Learn Python

### 重新實現銀行戶頭管理

In [2]:
class Account:
    def __init__(self, username, first_balance=200, **kwargs):
        self.name = username
        self.__balance = first_balance    # 這是 "私有變數" ，但實際上還是有方法可以存取

    def check_balance(self):
        print(""" Current Account Balance: {}""".format(self.__balance))

    def deposit(self, deposit):
        self.__balance += deposit
        print("""
           Deposit Money: {}
         Account Balance: {}""".format(deposit, self.__balance))

    def withdraw(self, withdraw):
        self.__balance -= withdraw
        print(
        """
          Withdraw Money: {} 
         Account Balance: {}""".format(withdraw, self.__balance))

account = Account('Jacky')
account.check_balance()
account.deposit(5000)
account.withdraw(600)
print()
account.balance = 0
account.check_balance()
print()
account._Account__balance = 0
account.check_balance()

 Current Account Balance: 200

           Deposit Money: 5000
         Account Balance: 5200

          Withdraw Money: 600 
         Account Balance: 4600

 Current Account Balance: 4600

 Current Account Balance: 0


# 

### 用 <font color="#ff0000">@property</font> 為銀行帳戶增加<font color="#ff0000">登入機制</font>

In [3]:
# 將明文密碼進行加密的函式庫
from werkzeug.security import generate_password_hash, check_password_hash

            
# 用 dict 來模擬會員資料庫
global AccountDB    # 設定全域變數
AccountDB = {}


class Account:
    global AccountDB
    
    def __init__(self, userid, password="0000", first_money=200, **kwargs):
        # 輸入會員資料
        self.lock = False
        self.userid = userid
        if self.userid not in AccountDB.keys():
            AccountDB[self.userid] = {}
            AccountDB[self.userid]['userid'] = self.userid
            AccountDB[self.userid]['password_hash'] = generate_password_hash(password)
            AccountDB[self.userid]['balance'] = first_money
            print(self.userid,'您好\n\n誠摯歡迎您的加入。')
                
    def __call__(self, password):
        self.lock = self.verify_password(password)
        if self.lock:
            print('\n登入成功。')
            self.__check__account_info()
        else: print('\n密碼錯誤!')
        return self.lock
                
    def __check__account_info(self, **kwargs):
        if len(kwargs) > 0:
            for key, val in kwargs.items():
                AccountDB[self.userid][key] = val
        if (len(AccountDB[self.userid]) <= 5 
                and 'address' not in AccountDB[self.userid]
                and 'phone' not in AccountDB[self.userid]):
            print('您的基本資料尚未完成!!!')
        
    def get_account_info(self):
        if self.lock:
            for key, val in AccountDB[self.userid].items():
                print('%-20s:'%key, val)
        
    def set_account_info(self, **kwargs):
        if self.lock:
            for key, val in kwargs.items():
                AccountDB[self.userid][key] = val
            for key, val in AccountDB[self.userid].items():
                print('%-20s:'%key, val)
        
    #----- 導入密碼功能並進行封裝 -----#
    @property
    def password(self):
        raise AttributeError('password is not readable attribute ...')

    @password.setter
    def password(self, password):
        AccountDB[self.userid]['password_hash'] = generate_password_hash(password)

    def verify_password(self, password):
        return check_password_hash(AccountDB[self.userid]['password_hash'], password)

### 新建帳戶

In [4]:
account = Account('ioplab', '0000')
account('0000')
account.get_account_info()

ioplab 您好

誠摯歡迎您的加入。

登入成功。
您的基本資料尚未完成!!!
userid              : ioplab
password_hash       : pbkdf2:sha256:150000$laUBeCB8$887a7d77ef407dbd04cf932f9915532a3889aff0a210dd44fd4f3c29dd85e472
balance             : 200


<h3><font color="#ff0000">@property</font> 運行結果</h3>

In [5]:
print('Verify first password: ', account.verify_password('0000'))
print()

# 更改密碼
print("更改密碼")
account.password = '123'
print('Verify old password: ', account.verify_password('0000'))
print('Verify new password: ', account.verify_password('123'))
print()

print("禁止讀取密碼")
account.password

Verify first password:  True

更改密碼
Verify old password:  False
Verify new password:  True

禁止讀取密碼


AttributeError: password is not readable attribute ...

In [8]:
print('重新登入 (使用舊密碼)')
account('0000')
print('\n-------\n')
print('重新登入 (使用新密碼)')
account('123')

重新登入 (使用舊密碼)

密碼錯誤!

-------

重新登入 (使用新密碼)

登入成功。
您的基本資料尚未完成!!!


True

In [9]:
print('---------- 新增資料前 ----------')
account.get_account_info()

print()
print('---------- 新增資料後 ----------')
account.set_account_info(
    address="台北市文山區116汀州路四段88號",
    phone="(02)7749-6736"
)

---------- 新增資料前 ----------
userid              : ioplab
password_hash       : pbkdf2:sha256:150000$lX3tzBHi$a314ea5e3a880a4bec7d3ff03c7bf2e66005e62e1310e97646d9f1e4aa0c45aa
balance             : 200

---------- 新增資料後 ----------
userid              : ioplab
password_hash       : pbkdf2:sha256:150000$lX3tzBHi$a314ea5e3a880a4bec7d3ff03c7bf2e66005e62e1310e97646d9f1e4aa0c45aa
balance             : 200
address             : 台北市文山區116汀州路四段88號
phone               : (02)7749-6736


###### 

### 用銀行帳戶例子說明 "繼承"

In [10]:
class BasicAccount(Account):
    def __init__(self, userid):
        super().__init__(userid)
    
    #----- 原先的方法 -----#
    def check_balance(self):
        if self.lock:
            print(
            """ Current Account Balance: {}""".format(AccountDB[self.userid]['balance']))

    def deposit(self, deposit):
        if self.lock:
            AccountDB[self.userid]['balance'] += deposit
            print(
            """ 
               Deposit Money: {}
             Account Balance: {}""".format(deposit, AccountDB[self.userid]['balance']))

    def withdraw(self, withdraw):
        if self.lock:
            AccountDB[self.userid]['balance'] -= withdraw
            print(
            """ 
              Withdraw Money: {} 
             Account Balance: {}""".format(withdraw, AccountDB[self.userid]['balance']))

In [11]:
basic_account = BasicAccount('ioplab')
basic_account('123')
basic_account.check_balance()
basic_account.deposit(5000)
basic_account.withdraw(600)
print()
basic_account.get_account_info()


登入成功。
 Current Account Balance: 200
 
               Deposit Money: 5000
             Account Balance: 5200
 
              Withdraw Money: 600 
             Account Balance: 4600

userid              : ioplab
password_hash       : pbkdf2:sha256:150000$lX3tzBHi$a314ea5e3a880a4bec7d3ff03c7bf2e66005e62e1310e97646d9f1e4aa0c45aa
balance             : 4600
address             : 台北市文山區116汀州路四段88號
phone               : (02)7749-6736


# 

###  模擬線上 ATM 功能

In [13]:
# 密碼層級的亂數函式
import secrets
import string    # 獲得 字串工具

# 產生密碼層級的亂數
# 含有 小寫英文、三個以上大寫英文 及 兩個數字
def generator_captcha():
    # 獲得 小寫英文 大寫英文 數字
    alphabet = string.ascii_letters + string.digits
    while True:
        password = ''.join(secrets.choice(alphabet) for _ in range(10))
        if (any(c.islower() for c in password)
                and sum(c.isupper() for c in password) == 3
                and sum(c.isdigit() for c in password) > 2):
            return password


while True:
    userid = input("請輸入帳號: [Enter:停止輸入]\n")
    if userid == "":
        break
    basic_account = BasicAccount(userid)

    for i in range(4):
        if i == 3:
            break
        password = input("請輸入密碼：\n")
        captcha = generator_captcha()
        print("驗證碼",captcha)
        usr_captcha = input("請輸入驗證碼: [需分大小寫] [Enter:停止輸入]\n")
        if usr_captcha != captcha:
            print('驗證失敗!!!')
        else:
            if basic_account(password):
                break

    if basic_account.lock:
        print(
            """
            服務項目:
            0  -  餘額查詢
            1  -  存款
            2  -  提款
            8  -  更改密碼
            9  -  離開系統
            """
        )
        service_id = int(input("請選擇服務項目: [Enter:停止輸入]\n"))
        if service_id == 0:
            basic_account.check_balance()
        elif service_id == 1:
            money = int(input("請輸入金額: [Enter:停止輸入]\n"))
            basic_account.deposit(money)
        elif service_id == 2:
            money = int(input("請輸入金額: [Enter:停止輸入]\n"))
            basic_account.withdraw(money)
        elif service_id == 8:
            print("更改密碼")
            old_password = input("請輸入舊密碼: ")
            if not basic_account.verify_password(old_password):
                break
            password = ""
            while not basic_account.verify_password(password):
                password = old_password
                basic_account.password = input("請輸入新密碼: ")
                password = input("請確認新密碼: ")
        else:
            pass
    break

請輸入帳號: [Enter:停止輸入]
 ioplab
請輸入密碼：
 123


驗證碼 06Snv25QPp


請輸入驗證碼: [需分大小寫] [Enter:停止輸入]
 06Snv25QPp



登入成功。

            服務項目:
            0  -  餘額查詢
            1  -  存款
            2  -  提款
            8  -  更改密碼
            9  -  離開系統
            


請選擇服務項目: [Enter:停止輸入]
 0


 Current Account Balance: 4600
